In [2]:
import numpy as np

def f1(x):
    return abs(x[4] + x[3] - 1.803)

def f2(x):
    return abs((x[1] + x[2]) * x[4] + 6.19116 * x[3] - 1.803 * (1.497 + 0.035))

def f3(x):
    return abs(x[5] + x[3] - 0.328)

def f4(x):
    return abs(0.28801 * x[5] - x[1] * x[2] * x[4])

def f5(x):
    return abs((-6.19116 * x[0] + x[0] * x[2] + x[1] * x[4] - x[2] * x[4]) * x[5] + x[0] * x[2] * x[4])

def f6(x):
    return abs(1.571 * x[6] + x[3] - 1.803)

def f7(x):
    return abs(x[7] - 0.000856 * x[6]**2)

def f8(x):
    return abs((x[4] - x[0]) * x[8] - x[0] * x[4])

def f9(x):
    return abs(x[8] - 377 * x[1] * x[7])

LB = np.array([-0.5, -1, -1, -1, 1, -1, -1, 0, -1])
UB = np.array([0.5, 1, 1, 1, 2, 1, 1, 1, 1])

In [3]:
def penalty(x):
    penalties = [f2(x), f3(x), f4(x), f5(x), f6(x), f7(x), f8(x)]
    return sum(p**2 for p in penalties)

def objective(x):
    return f1(x) + penalty(x)

def initialize_population(population_size, num_genes):
    return np.random.uniform(low=LB, high=UB, size=(population_size, num_genes))

def tournament_selection(population, tournament_size):
    idx = np.random.choice(len(population), tournament_size, replace=False)
    tournament = population[idx]
    return tournament[np.argmin([objective(individual) for individual in tournament])]

def arithmetic_crossover(parent1, parent2, crossover_rate):
    if np.random.rand() < crossover_rate:
        offspring1, offspring2 = parent1.copy(), parent2.copy()
        alpha = np.random.uniform(-0.5, 1.5, size=len(parent1))
        offspring1 = alpha * parent1 + (1 - alpha) * parent2
        offspring2 = alpha * parent2 + (1 - alpha) * parent1
        return offspring1, offspring2
    else:
        return parent1, parent2

def gaussian_mutation(offspring, mutation_rate, mutation_scale):
    for idx in range(len(offspring)):
        if np.random.rand() < mutation_rate:
            random_value = np.random.normal(loc=0, scale=mutation_scale, size=1)
            offspring[idx] += random_value
            offspring[idx] = np.clip(offspring[idx], LB[idx], UB[idx])
    return offspring

In [4]:
def run_genetic_algorithm(num_runs, population_size, num_genes, num_generations, tournament_size,
                          crossover_rate, mutation_rate, mutation_scale):
    all_metrics = []

    for run in range(num_runs):
        population = initialize_population(population_size, num_genes)
        best_fitness = np.inf
        best_solution = None
        fitnesses = []

        for generation in range(num_generations):
            fitness = np.array([objective(individual) for individual in population])
            fitnesses.append(fitness)

            min_fitness_idx = np.argmin(fitness)
            if fitness[min_fitness_idx] < best_fitness:
                best_fitness = fitness[min_fitness_idx]
                best_solution = population[min_fitness_idx]

            parents = [tournament_selection(population, tournament_size) for _ in range(population_size)]

            new_population = []
            for i in range(0, population_size, 2):
                parent1, parent2 = parents[i], parents[i + 1]
                offspring1, offspring2 = arithmetic_crossover(parent1, parent2, crossover_rate)
                offspring1 = gaussian_mutation(offspring1, mutation_rate, mutation_scale)
                offspring2 = gaussian_mutation(offspring2, mutation_rate, mutation_scale)
                new_population.extend([offspring1, offspring2])

            population = np.array(new_population)[:population_size]

        run_metrics = {
            'Run': run + 1,
            'Best Fitness': np.min(fitnesses),
            'Worst Fitness': np.max(fitnesses),
            'Mean Fitness': np.mean(fitnesses),
            'Std Dev Fitness': np.std(fitnesses),
            'Median Fitness': np.median(fitnesses)
        }
        all_metrics.append(run_metrics)

        print(f"Run {run + 1}: Best Fitness = {best_fitness}, Best Solution = {best_solution}")

    print("\nMetrics Table:")
    print("-" * 110)
    print(f"{'Run':<5} | {'Best Fitness':<15} | {'Worst Fitness':<15} | {'Mean Fitness':<15} | {'Std Dev Fitness':<20} | {'Median Fitness':<15}")
    print("-" * 110)
    for result in all_metrics:
        print(f"{result['Run']:<5} | {result['Best Fitness']:<15.6f} | {result['Worst Fitness']:<15.6f} | {result['Mean Fitness']:<15.6f} | {result['Std Dev Fitness']:<20.6f} | {result['Median Fitness']:<15.6f}")
    print("-" * 110)

    all_best_fitnesses = np.array([result['Best Fitness'] for result in all_metrics])
    all_worst_fitnesses = np.array([result['Worst Fitness'] for result in all_metrics])
    all_mean_fitnesses = np.array([result['Mean Fitness'] for result in all_metrics])
    all_std_fitnesses = np.array([result['Std Dev Fitness'] for result in all_metrics])
    all_median_fitnesses = np.array([result['Median Fitness'] for result in all_metrics])

    overall_best_fitness = np.min(all_best_fitnesses)
    overall_worst_fitness = np.max(all_worst_fitnesses)
    overall_mean_fitness = np.mean(all_mean_fitnesses)
    overall_std_fitness = np.sqrt(np.sum(np.square(all_std_fitnesses)) / len(all_std_fitnesses))
    overall_median_fitness = np.median(all_median_fitnesses)

    print("-" * 110)
    print(f"{'Overall':<5} | {overall_best_fitness:<15.6f} | {overall_worst_fitness:<15.6f} | {overall_mean_fitness:<15.6f} | {overall_std_fitness:<20.6f} | {overall_median_fitness:<15.6f}")
    print("-" * 110)

In [5]:
num_runs = 30
population_size = 200
num_genes = len(LB)
num_generations = 500
tournament_size = 5
crossover_rate = 0.9
mutation_rate = 0.1
mutation_scale = 0.05

In [6]:
run_genetic_algorithm(num_runs, population_size, num_genes, num_generations, tournament_size,
                      crossover_rate, mutation_rate, mutation_scale)

C:\Users\pedro\AppData\Local\Temp\ipykernel_7732\1804371508.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  offspring[idx] += random_value


Run 1: Best Fitness = 1.69093730426361e-07, Best Solution = [-9.38896467e-02 -3.94194589e-01  1.00551154e-01  5.07595521e-01
  1.29540448e+00 -1.79480700e-01  8.24567656e-01  6.17633231e-04
 -8.74891631e-02]
Run 2: Best Fitness = 0.00015293255976017535, Best Solution = [ 0.06382618  0.28229112 -0.00944769  0.3834333   1.41956326 -0.05052947
  0.90367379 -0.00144914  0.06746483]
Run 3: Best Fitness = 1.9156650774978584e-06, Best Solution = [-5.65190835e-02 -1.76583840e-01  1.45379598e-01  4.52964744e-01
  1.35003526e+00 -1.24540787e-01  8.59380386e-01  6.21724900e-04
 -5.41329734e-02]
Run 4: Best Fitness = 5.243446463263769e-05, Best Solution = [ 7.01735037e-02  2.75450510e-01 -2.76011499e-02  3.89433436e-01
  1.41356664e+00 -5.93022383e-02  8.99247389e-01  3.21284453e-05
  7.45019516e-02]
Run 5: Best Fitness = 6.477072854822174e-06, Best Solution = [ 1.72830913e-01  1.05622114e-01 -3.86648322e-01  5.04990532e-01
  1.29800948e+00 -1.76763428e-01  8.26292681e-01  2.12030959e-04
  1.99441